In [1]:
import pandas as pd
import numpy as np

# data science library
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics

#tree vizualization
from sklearn.tree import export_graphviz
from IPython.display import Image

#plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder

In [2]:
import pyarrow.feather as pf
import sklearn
arrtab = pf.read_table("./data_1.arr")
random_dataset = arrtab.to_pandas(timestamp_as_object=True)


In [3]:
random_dataset

,fail,lotWaferDie,XRel,YRel,XSize,YSize,DefArea,DefSize,dieRow,dieCol,xidx,yidx
0,0,W01-D105_37,1247042,730428,99130,98919,9.805876e+09,198050,105.0,37.0,5.0,-52
1,0,W01-D12_22,1667562,606900,66087,65946,4.358167e+09,132037,12.0,22.0,-10.0,41
2,1,W01-D62_28,222355,610062,99130,131892,1.307450e+10,247960,62.0,28.0,-4.0,-9
3,0,W01-D80_4,1981200,759210,12990,6490,8.431000e+07,6490,80.0,4.0,-28.0,-27
4,0,W01-D8_25,214360,236200,19490,19470,3.793900e+08,19470,8.0,25.0,-7.0,45
...,...,...,...,...,...,...,...,...,...,...,...,...
242393,1,W25-D86_46,2054940,296560,165217,197838,3.268625e+10,362968,86.0,46.0,14.0,-33
242394,1,W25-D87_15,2137090,51580,12990,6490,8.431000e+07,6490,87.0,15.0,-17.0,-34
242395,0,W25-D96_21,462610,708200,33043,65946,2.179084e+09,98986,96.0,21.0,-11.0,-43
242396,0,W25-D99_19,1026344,808760,99130,98919,9.805876e+09,224044,99.0,19.0,-13.0,-46


In [4]:
random_dataset.notnull().sum() #there are no null values

fail           200279
lotWaferDie    200279
XRel           200279
YRel           200279
XSize          200279
YSize          200279
DefArea        200279
DefSize        200279
dieRow         200279
dieCol         200279
xidx           200279
yidx           200279
dtype: int64

In [5]:
random_dataset.rename(columns={'fail':'label'}, inplace=True) #rename the fail as label because we are using clasification

In [6]:
y=random_dataset['label']

In [7]:
y

0         0
1         0
2         1
3         0
4         0
         ..
242393    1
242394    1
242395    0
242396    0
242397    0
Name: label, Length: 200279, dtype: int32

In [8]:
feature_cols = [col for col in random_dataset.columns if col != 'label']

In [9]:
random_dataset.dtypes

label            int32
lotWaferDie     object
XRel             int32
YRel             int32
XSize            int32
YSize            int32
DefArea        float64
DefSize          int32
dieRow         float64
dieCol         float64
xidx           float64
yidx             int32
dtype: object

In [10]:
obj_df = random_dataset.select_dtypes(include=['object']).copy()
obj_df

,lotWaferDie
0,W01-D105_37
1,W01-D12_22
2,W01-D62_28
3,W01-D80_4
4,W01-D8_25
...,...
242393,W25-D86_46
242394,W25-D87_15
242395,W25-D96_21
242396,W25-D99_19


In [11]:
encoder = OrdinalEncoder()
random_dataset['lotWaferDie'] = encoder.fit_transform(random_dataset[['lotWaferDie']])


In [12]:
X=random_dataset[feature_cols]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [14]:
clf = DecisionTreeClassifier()


In [15]:
clf = clf.fit(X_train, y_train)

In [16]:
y_pred_train = clf.predict(X_train)

In [17]:
print('Train accurcy:', metrics.accuracy_score(y_train,y_pred_train))

Train accurcy: 0.997689885426306


In [18]:
y_pred = clf.predict(X_test)

In [19]:
print(f'Test Accuracy:  {np.round(metrics.accuracy_score(y_test, y_pred),2)}')

Test Accuracy:  0.87


In [20]:
from sklearn.metrics import f1_score

In [21]:
f1 = f1_score(y_test, y_pred)

In [22]:
f1

0.5519771708112515

In [23]:
print(f"y_test distribution: {np.bincount(y_test)}")
print(f"y_pred distribution: {np.bincount(y_pred)}")

y_test distribution: [42814  7256]
y_pred distribution: [42608  7462]


In [24]:
arrtab_test = pf.read_table("./data_1_test_no_target.arr")
test_dataset = arrtab_test.to_pandas(timestamp_as_object=True)

In [25]:
encoder = OrdinalEncoder()
test_dataset['lotWaferDie'] = encoder.fit_transform(test_dataset[['lotWaferDie']])

In [26]:
test_dataset = test_dataset[X.columns]

In [27]:
y_pred_test = clf.predict(test_dataset)

In [28]:
predictions_df = pd.DataFrame({'Predicted_Label': y_pred_test})

# Save to CSV
predictions_df.to_csv("predictions.csv", index=False)

print("Predictions saved to test_predictions.csv")

Predictions saved to test_predictions.csv


In [29]:
y_pred_test

array([1, 1, 0, ..., 0, 1, 1])